In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-120481")
exp = Experiment(workspace=ws, name="quick-starts-ws-120481")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-120481
Azure region: southcentralus
Subscription id: b19c9c81-5f59-4537-b3c0-c1beb163ec22
Resource group: aml-quickstarts-120481


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

from azureml.core.compute_target import ComputeTargetException

cluster_name = "pipeline-clust"

try:
	compute_target = ComputeTarget(workspace=ws, name=cluster_name)
	print('Found existing cluster, use it.')
except ComputeTargetException:
	compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                       	vm_priority = 'lowpriority',
                                                       	max_nodes=3)
	compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
	compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)



Found existing cluster, use it.


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os


# Specify parameter sampler
params = {
	"C": uniform(0.0, 1.0),
	"max_iter": choice(20, 60, 100, 140)
}

ps =  RandomParameterSampling(params)
### YOUR CODE HERE ###

# Specify a Policy
policy =    BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=5) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    print("Creating new directory")
    os.mkdir("./training")
else:
    print("Directory already existed")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
	    source_directory="./training",
	    compute_target=compute_target,
	    entry_script="train.py"
)   ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
	estimator=est,
	hyperparameter_sampling=ps,
	policy=policy,
	primary_metric_name="Accuracy",
	primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
	max_concurrent_runs=4,
	max_total_runs=10*len(params))      ### YOUR CODE HERE ###

Directory already existed


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()


WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
joblib.dump(value=best_hyperdrive_run, filename=os.path.join('./training', 'best_hyperdrive_model.joblib'))


['./training/best_hyperdrive_model.joblib']

In [6]:
best_run_metrics = best_hyperdrive_run.get_metrics()
parameter_values = best_hyperdrive_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n C:', parameter_values[1])
print('\n max_iter:', parameter_values[3])


AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimted_files(url)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds) ### YOUR DATA OBJECT HERE ###)
x['label'] = y

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric= 'accuracy',
    training_data= x,
    label_column_name= 'label',
    n_cross_validations= 5)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)
RunDetails(hyperdrive_run).show()


In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_run, fitted_automl_model = automl_run.get_output()
print(fitted_automl_model)
